<a href="https://colab.research.google.com/github/CefasRepRes/shipwreck-oil-detection-colab/blob/main/notebooks/Cloud_Based_Analysis_of_Oil_Spills_from_Wrecks_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cloud Based Analysis of Oil Spills from Wrecks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Set-up of workspace
Import Earth Engine API (`import ee`).
Additionally, import useful python packages from examplar scripts (need to remove redundant ones after)

In [ ]:
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd                                                             # Useful package to read in csv's etc...
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline



Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. Authorisation last a week.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

### Interactive map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map

In [ ]:
# Import the Folium library.
import folium                                                                   # Used for Interactive mapping

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,                                                                # Adds the layer name to the radio buttons rather than web address
    overlay = True,                                                             # Allow layers to overlay upon each
    control = True                                                              # Allows the user to turn on and off layers
  ).add_to(self)                                                                # Allows layers to be added after this base layer

# Add EE drawing method to folium, to be able to add other layers to this base map (see examplar below)
folium.Map.add_ee_layer = add_ee_layer                                         

from folium.plugins import Draw                                                 # Toolbox for adding the toolbar
draw = Draw(export=True)                                                        # Adds toolbars to folium plots


Examplar interactive map set to east anglia with a DEM

In [ ]:
# Set visualization parameters of DEM
vis_params = {
  'min': 0,                                                                     # value for first colour
  'max': 3000,                                                                  # last value for colour
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}                # Colours for the scale

# Create a folium map object, add zoomed area (Cefas)
my_map = folium.Map(location=[52.5, 2], zoom_start=5)

my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')               # Add the elevation model to the map object
my_map.add_child(folium.LayerControl())                                         # Add a layer control panel to the map
draw.add_to(my_map)                                                             # Add the draw panel

# Display the map
display(my_map)

## A Sentinel-1 image
The images are massive so want a spatial subset. A convenient way the area of interest (AOI) is to use the geojson.io website, from which we can cut and paste the corresponding [`GeoJSON`](https://geojson.io/#map=2/20/0) object description.

In [ ]:
geoJSON ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              0.7009966116023918,
              53.53216593286186
            ],
            [
              0.7009966116023918,
              52.57145958507567
            ],
            [
              2.3863359635554957,
              52.57145958507567
            ],
            [
              2.3863359635554957,
              53.53216593286186
            ],
            [
              0.7009966116023918,
              53.53216593286186
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

Note that the last and first corners are identical, indicating closure of the polygon. We have to bore down into the GeoJSON structure to get the geometry coordinates, then create an ee.Geometry() object:

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

Next, we filter the S1 archive to get an image over the AOI in NOV 2021. 
The orbit number or whether we want the ASCENDING or DESCENDING node not defined so need to look into this further. 
If we don't specify the instrument mode or resolution, we get IW (interferometric wide swath) mode and 10 X 10 m<sup>2</sup> pixels by default. For convenience we grab both decibel and float versions:

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2021-11-01'), ee.Date('2021-11-27')) 
                       .first())
                       #.clip(aoi))_
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2021-11-01'), ee.Date('2021-11-27')) 
                       .first())
                       #.clip(aoi))

Notice that we have clipped the images to our aoi so as not to work with the entire swath. To confirm that we have an image, we list its band names, fetching the result from the GEE servers with the *getInfo()* class method:

In [ ]:
ffa_db.bandNames().getInfo()

### Interactive map with the data
This is fine, but a little boring. We can use folium to project onto a map for geographical context. The folium Map() constructor wants its location keyword in long-lat rather than lat-long, so we do a list reverse in the first line:

In [ ]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=8)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())
WarMahtar = ee.Geometry.Point([-2.148451, 52.606052]);
# Display the map.
display(m)

### Extracting statistics from the sentinel Data
good [`userguide`](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1) here

## Interactive way to explore Sentinel data usign the GEE Sentinel-1 explorer widget
Firstly instal various bits to make to query the data interactively

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
!pip install -q ipyleaflet
!git clone https://github.com/mortcanty/eesarseq
%cd /content/eesarseq/src
%run setup install

In [ ]:
# GEE Sentinel-1 explorer widget
from eesar.application import run
run()

It is a bit clunky and not well refined but good to play with
see the good [`userguide`](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-4) on explanation of various buttons

In [ ]:
geoJSON ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              57.63547188262015,
              -20.333291895854103
            ],
            [
              57.63547188262015,
              -20.52842688301594
            ],
            [
              57.95189904577262,
              -20.52842688301594
            ],
            [
              57.95189904577262,
              -20.333291895854103
            ],
            [
              57.63547188262015,
              -20.333291895854103
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2020-07-26'),ee.Date('2020-08-15'))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())
timestamplist

In [ ]:
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

im_list.length().getInfo()

In [ ]:
def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=13)
rgb_images = (ee.Image.rgb(vv_list.get(1), vv_list.get(2), vv_list.get(3))
              .log10().multiply(10))
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
draw.add_to(mp)
mp.add_child(folium.LayerControl())